In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

# Round 2

In [2]:
# Reading files
data = pd.read_csv("./files_for_lab/csv_files/marketing_customer_analysis.csv")

In [3]:
# 1. Show the dataframe shape.
print(data.shape)

(10910, 26)


In [4]:
# 2. Standardize header names.

# I first drop the old index column
data = data.drop(['Unnamed: 0'],axis=1)

In [5]:
data.columns = list(map(lambda x: x.lower(), data.columns))
data.columns = data.columns.str.replace(' ','_')

In [6]:
data.select_dtypes(np.number)

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,4809.216960,48029,61,7.0,52,0.0,9,292.800000
1,2228.525238,0,64,3.0,26,0.0,1,744.924331
2,14947.917300,22139,100,34.0,31,0.0,2,480.000000
3,22332.439460,49078,97,10.0,3,0.0,2,484.013411
4,9025.067525,23675,117,NaN,31,NaN,7,707.925645
...,...,...,...,...,...,...,...,...
10905,15563.369440,0,253,NaN,40,NaN,7,1214.400000
10906,5259.444853,61146,65,7.0,68,0.0,6,273.018929
10907,23893.304100,39837,201,11.0,63,0.0,2,381.306996
10908,11971.977650,64195,158,0.0,27,4.0,6,618.288849


In [7]:
data.select_dtypes(object)

,customer,state,response,coverage,education,effective_to_date,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size,vehicle_type
0,DK49336,Arizona,No,Basic,College,2/18/11,Employed,M,Suburban,Married,Corporate Auto,Corporate L3,Offer3,Agent,Four-Door Car,Medsize,NaN
1,KX64629,California,No,Basic,College,1/18/11,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer4,Call Center,Four-Door Car,Medsize,NaN
2,LZ68649,Washington,No,Basic,Bachelor,2/10/11,Employed,M,Suburban,Single,Personal Auto,Personal L3,Offer3,Call Center,SUV,Medsize,A
3,XL78013,Oregon,Yes,Extended,College,1/11/11,Employed,M,Suburban,Single,Corporate Auto,Corporate L3,Offer2,Branch,Four-Door Car,Medsize,A
4,QA50777,Oregon,No,Premium,Bachelor,1/17/11,Medical Leave,F,Suburban,Married,Personal Auto,Personal L2,Offer1,Branch,Four-Door Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,FE99816,Nevada,No,Premium,Bachelor,1/19/11,Unemployed,F,Suburban,Married,Personal Auto,Personal L1,Offer3,Web,Luxury Car,Medsize,A
10906,KX53892,Oregon,No,Basic,College,1/6/11,Employed,F,Urban,Married,Personal Auto,Personal L3,Offer2,Branch,Four-Door Car,Medsize,A
10907,TL39050,Arizona,No,Extended,Bachelor,2/6/11,Employed,F,Rural,Married,Corporate Auto,Corporate L3,Offer1,Web,Luxury SUV,Medsize,NaN
10908,WA60547,California,No,Premium,College,2/13/11,Employed,F,Urban,Divorced,Personal Auto,Personal L1,Offer1,Branch,SUV,Medsize,A


In [8]:
# check and deal with NaN
#data[data.isna().any(axis=1)] 
# It seems we have a lot of NaNs. Most of them come from vehicle_type, we're missing 
# that data for half of the rows so I think I will just drop the column
data=data.drop(['vehicle_type'],axis=1)
# I'm not sure if I will need it later. 

In [9]:
data.isna().sum()

customer                           0
state                            631
customer_lifetime_value            0
response                         631
coverage                           0
education                          0
effective_to_date                  0
employmentstatus                   0
gender                             0
income                             0
location_code                      0
marital_status                     0
monthly_premium_auto               0
months_since_last_claim          633
months_since_policy_inception      0
number_of_open_complaints        633
number_of_policies                 0
policy_type                        0
policy                             0
renew_offer_type                   0
sales_channel                      0
total_claim_amount                 0
vehicle_class                    622
vehicle_size                     622
dtype: int64

In [10]:
nan_columns = ['state','response','number_of_open_complaints','vehicle_class','vehicle_size']

for column in nan_columns:
    data[column]=data[column].fillna(data[column].mode()[0])
    
# Around 80% responses are no so I will fill nulls with No

# California is just around 30% of the states we have, but it is
# the most frequent (mode) and so I will fill na values with CA

# For open complaints I'l lfill with 0 since it's the majority by far

# For vehice class I will fill with Four-Door Car

# For vehicle_size I will also fill with the mode Medsize

# For months_since_last_claim I'll replace with the mean
data['months_since_last_claim']=data['months_since_last_claim'].fillna(data['months_since_last_claim'].mean())


In [11]:
# converting to datetime format
data['effective_to_date']=pd.to_datetime(data['effective_to_date'])

In [12]:
#auxiliar function for the month extraction
def extract_month(date):
   months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
   return months[date.month-1]


In [13]:
# Creating "month" attribute through the extract_month function
data['month']=data['effective_to_date'].apply(extract_month)

# Easier way
# data['month']=data['effective_to_date'].dt.month_name()

In [14]:
# filter only for January, February and March
data[data['month'].isin(['January','February','March'])]
# I think we only have data for Jan and Feb since the number of rows looks the same. 

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,month
0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,48029,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,February
1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,0,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,January
2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,22139,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,February
3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,49078,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,January
4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,23675,...,0.0,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,2011-01-19,Unemployed,F,0,...,0.0,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,January
10906,KX53892,Oregon,5259.444853,No,Basic,College,2011-01-06,Employed,F,61146,...,0.0,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,January
10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2011-02-06,Employed,F,39837,...,0.0,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,February
10908,WA60547,California,11971.977650,No,Premium,College,2011-02-13,Employed,F,64195,...,4.0,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,February


# bonus function

In [ ]:
file_location = "./files_for_lab/csv_files/marketing_customer_analysis.csv"
def bonus(file_loc):
    
    #auxiliar function for the month extraction
    def extract_month(date):
        months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
        return months[date.month-1]

    # reading file
    data = pd.read_csv(file_loc)
    
    print('Printing datashape'),print(data.shape)
    
    #dropping old index
    data = data.drop(['Unnamed: 0'],axis=1)
    
    #to snake_case
    data.columns = list(map(lambda x: x.lower(), data.columns))
    data.columns = data.columns.str.replace(' ','_')
    
    print('\nPrinting numerical data'),print(data.select_dtypes(np.number).head(3))
    print('\n\nPrinting categorical data'),print(data.select_dtypes(object).head(3))
    
    # I drop this column tho I'm not sure it's the right way to approach
    # the problem.
    data=data.drop(['vehicle_type'],axis=1)
    
    # Filling the categorical NaN with mode values
    categ_nan_cols = data.select_dtypes(object).isna().sum()>0 #list of booleans
    categ_nan_cols = categ_nan_cols[categ_nan_cols] #choosing only TRUE columns
    categ_nan_cols = list(categ_nan_cols.index.values) #getting the column names
    
    for column in categ_nan_cols:
        breakpoint()
        data[column] = data[column].fillna(data[column].mode()[0])
    
    # Filling the numerical NaN with median values
    num_nan_cols = data.select_dtypes(np.number).isna().sum()>0
    num_nan_cols = num_nan_cols[num_nan_cols]
    num_nan_cols = list(num_nan_cols.index.values)
    
    for column in num_nan_cols:
        data[column]=data[column].fillna(data[column].mean())
    
    #Converting to datetime
    data['effective_to_date']=pd.to_datetime(data['effective_to_date'])
    
    # Creating "month" attribute through the extract_month function
    data['month']=data['effective_to_date'].apply(extract_month)

        
    # Finally filtering data for Jan,Feb,March
    data = data[data['month'].isin(['January','February','March'])]
    return data

bonus(file_location)